In [1]:
from swift.llm import PtEngine, RequestConfig
from swift.utils import get_logger, seed_everything
logger = get_logger()

/home/debian/workspace/multi_model/ms-swift/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'librosa'

In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'


def _infer_model(pt_engine, system=None, messages=None, audios=None):
    seed_everything(42)
    request_config = RequestConfig(max_tokens=128, temperature=0)
    if messages is None:
        messages = []
        if system is not None:
            messages += [{'role': 'system', 'content': system}]
        messages += [{'role': 'user', 'content': '你好'}]
        resp = pt_engine.infer([{'messages': messages}], request_config=request_config)
        response = resp[0].choices[0].message.content
        messages += [{'role': 'assistant', 'content': response}]
        messages += [{'role': 'user', 'content': '<audio>这段语音说了什么'}]
    else:
        messages = messages.copy()
    if audios is None:
        audios = ['http://modelscope-open.oss-cn-hangzhou.aliyuncs.com/images/weather.wav']
    resp = pt_engine.infer([{'messages': messages, 'audios': audios}], request_config=request_config)
    response = resp[0].choices[0].message.content
    messages += [{'role': 'assistant', 'content': response}]
    logger.info(f'model: {pt_engine.model_info.model_name}, messages: {messages}')
    return response

In [ ]:

    
pt_engine = PtEngine('/mnt/nfs/models/Step-Audio-2-mini')
messages = [{'role': 'user', 'content': '<audio>Caption the audio.'}]
response = _infer_model(pt_engine, messages=messages)
pt_engine.default_template.template_backend = 'jinja'
response2 = _infer_model(pt_engine, messages=messages)
    

In [ ]:
assert response == response2 == "The audio contains a male voice speaking the phrase '今天天气真好呀' in Mandarin."
